In [ ]:
import sqlite3

<h1 align="left" style="font-size:20px"><b> General Methods </b></h1>

In [ ]:
def get_col_vals(_df, _search_col, _tot_val):
    lst_analysis = _df[_search_col]
    df_analysis = pd.DataFrame({'answer':lst_analysis.values})
    df_analysis_grp = df_analysis.groupby('answer').size().reset_index()
    df_analysis_grp = df_analysis_grp.rename(columns={"answer": "Answer", 0: "Count"})
    df_analysis_grp.insert(2, "Percentage", 100 * (df_analysis_grp['Count'] / _tot_val), True)
    df_analysis_grp.insert(3, "Total", _tot_val, True)
    return df_analysis_grp

In [ ]:
#https://docs.python.org/3/library/sqlite3.html
class Sqlite3Db:
    def __init__(self, name=None):
        self.conn = None
        self.cursor = None

        if name:
            self.open(name)
    
    def open(self, name):
        try:
            self.conn = sqlite3.connect(name)
            self.cursor = self.conn.cursor()
        except sqlite3.Error as e:
            print("Error connecting to database " + name + " with message:" + e.args[0] )
    
    def close(self):
        if self.conn:
            self.cursor.close()
            self.conn.close()

    def query(self,sqlStm):
        self.cursor.execute(sqlStm)

    #Using these magic methods (__enter__, __exit__) allows you to implement objects which can be used easily with the with statement.
    def __enter__(self):
        return self.conn
    
    def __exit__(self,exc_type,exc_value,traceback):
        self.close()